In [12]:
# ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.family'] = 'IPAPGothic'
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
import oandapyV20.endpoints.instruments as instruments
import datetime
from statistics import mean
from statistics import median
import calendar
import datetime
from math import floor
import time
import threading
import json
import random
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from joblib import dump, load
from sklearn import metrics
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
import feather
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit

In [3]:
# OANDA API v20の口座IDとAPIトークン
accountID = "101-009-12789646-001"
access_token = "1ab53171ce4413f408de6da28ea23089-6426c3d3cc947dba212b3766d1be45c6"
# OANDAのデモ口座へのAPI接続
api = API(access_token=access_token, environment="practice")

In [4]:
# APIから取得したレートをPandasのDataFrameへ
def to_dataframe(r):
    data = []
    for raw in r.response['candles']:
        data.append([raw['time'], raw['volume'], raw['mid']['o'], raw['mid']['h'], raw['mid']['l'], raw['mid']['c']])

    # リストからPandas DataFrameへ変換
    df = pd.DataFrame(data)
    df.columns = ['Time', 'Volume', 'Open', 'High', 'Low', 'Close']
    df['Time'] = pd.to_datetime(df['Time'])
    for col in ['Open', 'High', 'Low', 'Close']:
        df[col] = df[col].apply(float)
    return df

In [5]:
def get_data(count, gran, year, month, day, hour, minute, second):
    # OANDA API v20の口座IDとAPIトークン
    accountID = "101-009-12789646-001"
    access_token = "1ab53171ce4413f408de6da28ea23089-6426c3d3cc947dba212b3766d1be45c6"
    # OANDAのデモ口座へのAPI接続
    api = API(access_token=access_token, environment="practice")
    fmt = '%Y-%m-%dT%H:%M:00.000000Z'
    _from = datetime.datetime(year=year, month=month, day=day,
                              hour=hour, minute=minute, second=second).strftime(fmt)
    params = {
        "count": count,
        "granularity": gran,
        'from': _from,
#         'dailyAlignment': 0
    }
    r = instruments.InstrumentsCandles(instrument="USD_JPY", params=params)
    api.request(r)
    df = to_dataframe(r)
    return df

In [6]:
def get_data_super(start, end, gran):
    # OANDA API v20の口座IDとAPIトークン
    accountID = "101-009-12789646-001"
    access_token = "1ab53171ce4413f408de6da28ea23089-6426c3d3cc947dba212b3766d1be45c6"
    # OANDAのデモ口座へのAPI接続
    api = API(access_token=access_token, environment="practice")
    # xxxx-xx-xx～xxxx-xx-xx の期間のデータを取得してデータフレームを返す
    s_year, s_month, s_day, s_hour, s_minute, s_second = map(int, start.split('-'))
    e_year, e_month, e_day, e_hour, e_minute, e_second = map(int, end.split('-'))
    fmt = '%Y-%m-%dT%H:%M:00.000000Z'
    from_ = datetime.datetime(year=s_year, month=s_month, day=s_day,
                              hour=s_hour, minute=s_minute, second=s_second).strftime(fmt)
    to_ = datetime.datetime(year=e_year, month=e_month, day=e_day,
                            hour=e_hour, minute=e_minute, second=e_second).strftime(fmt)
    df = pd.DataFrame()
    # 5000個制限に引っかからなければこっちの処理
    try:
        params = {
            "granularity": gran,
            'from': from_,
            'to': to_
        }
        r = instruments.InstrumentsCandles(instrument="USD_JPY", params=params)
        api.request(r)
        df = to_dataframe(r)
    # 引っかかればこっちの処理
    except:
        i = True
        while i:
            df_tmp = get_data(count=5000, gran=gran, year=s_year, month=s_month, day=s_day,
                             hour=s_hour, minute=s_minute, second=s_second)
            df = pd.concat([df, df_tmp], ignore_index=True)
            index = df_tmp.tail(1).index[0]
            s_year = df_tmp.iat[index, 0].year
            s_month = df_tmp.iat[index, 0].month
            s_day = df_tmp.iat[index, 0].day
            s_hour = df_tmp.iat[index, 0].hour
            s_minute = (df_tmp.iat[index, 0].minute + 1) % 60
            s_second = df_tmp.iat[index, 0].second
#             print('{}-{}-{}-{}-{}-{}'.format(s_year, s_month, s_day, s_hour, s_minute, s_second))
            # 5000個ずつ取得してエンド超えたら止める
            if s_year == e_year:
                i = False
        # 止めたらはみ出した分をここで切り落とす
        for i in range(len(df.index) - 5000, len(df.index)):
            if df.iat[i, 0].year == e_year and df.iat[i, 0].month == e_month and df.iat[i, 0].day == e_day:
                df = df[0:i]
                break
    # ボリンジャーバンドを追加
    sigma = 1
    df['BB20,+{}'.format(sigma)] = df['BB20,-{}'.format(sigma)] = 0
    df['BB20,+{}'.format(sigma)] = df['BB20,+{}'.format(sigma)].apply(float)
    df['BB20,-{}'.format(sigma)] = df['BB20,-{}'.format(sigma)].apply(float)
    for i in range(20, len(df)):
        df.iat[i, 6] = round(mean(df['Close'][i-20:i]) + (np.std(df['Close'][i-20:i]) * sigma), 3)
        df.iat[i, 7] = round(mean(df['Close'][i-20:i]) - (np.std(df['Close'][i-20:i]) * sigma), 3)
    # シグナル追加
    df['Lシグナル'] = df['Sシグナル'] = 0
    for i in range(len(df)):
        if df.iat[i, 6] < df.iat[i, 5]:
            df.iat[i, 8] = 1
        if df.iat[i, 7] > df.iat[i, 5]:
            df.iat[i, 9] = 1
    # 移動平均追加
    df['SMA20'] = 0
    df['SMA20'] = df['SMA20'].apply(float)
    for i in range(20, len(df)):
        df.iat[i, 10] = round(mean(df['Close'][i-20:i]), 3)
    # BB と SMA で参照した最初の分を落とす
    df = df[20:]
    df.reset_index(drop=True, inplace=True)
    return df

In [7]:
# 訓練データ作る関数 (binary用)
def get_dataset_bc(df, n, m):
    LGB_df = pd.DataFrame()
    for i in range(1, n+1):
        diff_tmp = df.iloc[:, [2,3,4,5,6,7,10]].diff(i)*100
        diff_tmp['Volume'] = df.iloc[:, [1]].shift(i)
        diff_tmp['Lシグナル'] = df.iloc[:, [8]].shift(i)
        diff_tmp['Sシグナル'] = df.iloc[:, [9]].shift(i)
        LGB_df = pd.concat([LGB_df, diff_tmp], axis=1)
    LGB_df['y'] = df['Close'].diff(-10)*100
    LGB_df = LGB_df[n:(len(LGB_df)-n)]
    LGB_df_1 = LGB_df[LGB_df['y'] >= m].assign(target=1)
    LGB_df_2 = LGB_df.query('y < {}'.format(m)).assign(target=0)
    LGB_df_c = pd.concat([LGB_df_1, LGB_df_2], axis=0)
    # LGB_df_c.drop(['y'], axis=1, inplace=True)
    # LGB_df_c = LGB_df_c.sample(frac=1, random_state=1729)
    LGB_df_c.reset_index(drop=True, inplace=True)
    return LGB_df_c

In [8]:
# df = pd.read_csv('data/df.csv')
df = pd.read_csv('data/df_sma.csv')
# LGB_df = pd.read_csv('data/LGB_df_BB.csv')
# LGB_df = pd.read_csv('data/LGB_df_SMA.csv')
# df_10 = pd.read_csv('data/df_10.csv')
df_5 = pd.read_csv('data/df_5.csv')
df_2017 = pd.read_csv('data/df_2017.csv')

In [9]:
LGB_df_bc = get_dataset_bc(df, 50, 10)
# LGB_df_bc

In [40]:
LGB_df_bc.columns = map(str, range(502))
X = LGB_df_bc.drop(['500', '501'], axis=1).to_numpy()
y = LGB_df_bc[['501']].to_numpy().flatten()

In [41]:
lgbm_params = {
    'objective': 'binary',
    'metric': 'auc',
    'seed': 1729,
    'learning_rate': 0.05
}

KFold

In [57]:
KF = KFold(n_splits=10, shuffle=True, random_state=42)
KF.get_n_splits(X)

10

In [58]:
auc_l = []
for train_index, test_index in KF.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    X_train2, X_valid, y_train2, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=42)
    lgb_train = lgb.Dataset(X_train2, y_train2)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)
    LGB = lgb.train(lgbm_params, lgb_train,
                    # モデルの評価用データを渡す
                    valid_sets=lgb_eval,
                    # 最大で 1000 ラウンドまで学習する
                    num_boost_round=1000,
                    # 10 ラウンド経過しても性能が向上しないときは学習を打ち切る
                    early_stopping_rounds=10,
                    # うるさいので黙らせる
                    verbose_eval = False)
    y_pred = LGB.predict(X_test, num_iteration=LGB.best_iteration)
    auc = metrics.roc_auc_score(y_test, y_pred)
    print(auc)
    auc_l.append(auc)
print('mean: {}'.format(mean(auc_l)))

0.8089658538518746
0.7988209409417719
0.7807959636147674
0.7991233388995888
0.8056932888011916
0.7886183016752675
0.7935641659781111
0.8050697520276738
0.7869136914177702
0.7718052312157072
mean: 0.7939370528423724


In [59]:
# 3 KFold
# mean: 0.7836842971095238
# 10 KFold seed = 1729
# mean: 0.7936685896273451
# 10 KFold seed = 42
# mean: 0.7939370528423724

StratifiedKFold

In [69]:
SKF = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
SKF.get_n_splits(X, y)

10

In [70]:
auc_l = []
for train_index, test_index in SKF.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    X_train2, X_valid, y_train2, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=42)
    lgb_train = lgb.Dataset(X_train2, y_train2)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)
    LGB = lgb.train(lgbm_params, lgb_train,
                    # モデルの評価用データを渡す
                    valid_sets=lgb_eval,
                    # 最大で 1000 ラウンドまで学習する
                    num_boost_round=1000,
                    # 10 ラウンド経過しても性能が向上しないときは学習を打ち切る
                    early_stopping_rounds=10,
                    # うるさいので黙らせる
                    verbose_eval = False)
    y_pred = LGB.predict(X_test, num_iteration=LGB.best_iteration)
    auc = metrics.roc_auc_score(y_test, y_pred)
    print(auc)
    auc_l.append(auc)
print('mean: {}'.format(mean(auc_l)))

0.8005931497085041
0.7763550660619003
0.8065124482201068
0.7859698989810447
0.8031150237203107
0.8092663421589752
0.783071724506159
0.7747727567083356
0.794882625721018
0.7649178851346061
mean: 0.7899456920920961


StratifiedShuffleSplit

In [75]:
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=1729)
sss.get_n_splits(X, y)

10

In [76]:
auc_l = []
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    X_train2, X_valid, y_train2, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=42)
    lgb_train = lgb.Dataset(X_train2, y_train2)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)
    LGB = lgb.train(lgbm_params, lgb_train,
                    # モデルの評価用データを渡す
                    valid_sets=lgb_eval,
                    # 最大で 1000 ラウンドまで学習する
                    num_boost_round=1000,
                    # 10 ラウンド経過しても性能が向上しないときは学習を打ち切る
                    early_stopping_rounds=10,
                    # うるさいので黙らせる
                    verbose_eval = False)
    y_pred = LGB.predict(X_test, num_iteration=LGB.best_iteration)
    auc = metrics.roc_auc_score(y_test, y_pred)
    print(auc)
    auc_l.append(auc)
print('mean: {}'.format(mean(auc_l)))

0.7857756014498638
0.7808714945798012
0.7851930691073468
0.7919553475543327
0.7772772668742753
0.7905908421114124
0.7798869188754509
0.7963503296615326
0.7848703260026689
0.7736988805643239
mean: 0.7846470076781009


In [77]:
# sss = 3
# mean: 0.7839467217123373
# sss = 10
# mean: 0.7846470076781009